# Hyperparameter tuning
This notebook explores hyperparameter tuning. It uses the boston house price dataset built into Sklearn.

## Import

In [2]:
# Core libraries
import pandas as pd

# Sklearn processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Sklearn regression algorithms
from sklearn.neighbors import KNeighborsRegressor

# Sklearn regression model evaluation functions
from sklearn.metrics import r2_score

## Load data, split into X and y and scale data

In [3]:
# Load built-in sample data set
from sklearn.datasets import load_boston
boston = load_boston()

# Define the X (input) and y (target) features
X = boston.data
y = boston.target

# Rescale the input features
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(boston.data)

## Build a model with default hyperparameters

In [4]:
# Create an empty model
model = KNeighborsRegressor()

In [5]:
# Inspect the model's default hyperparameters:
model

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [5]:
# What hyperparameters can we tune?
model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

## Tune hyperparameters with grid search 

In [6]:
# Select an algorithm
algorithm = KNeighborsRegressor()

# Create 3 folds
seed = 13
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)

# Define our candidate hyperparameters
hp_candidates = [{'n_neighbors': [2,3,4,5,6], 'weights': ['uniform','distance']}]

# Search for best hyperparameters
grid = GridSearchCV(estimator=algorithm, param_grid=hp_candidates, cv=kfold, scoring='r2')
grid.fit(X, y)

# Get the results
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_)

0.7711643653097031
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=3, p=2,
          weights='distance')
{'n_neighbors': 3, 'weights': 'distance'}


### Get a full breakdown of the grid search

In [9]:
grid.cv_results_

C:\Users\llewe\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\llewe\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\llewe\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\llewe\Anaconda3\lib\site-packages\sklearn\util

{'mean_fit_time': array([0.00068895, 0.0006659 , 0.00066487, 0.        , 0.00033291,
        0.00033267, 0.        , 0.00033021, 0.        , 0.00066503]),
 'std_fit_time': array([0.00048801, 0.00047086, 0.00047013, 0.        , 0.00047081,
        0.00047047, 0.        , 0.00046699, 0.        , 0.00047025]),
 'mean_score_time': array([0.00165955, 0.00132926, 0.00099691, 0.00166241, 0.00165264,
        0.00099738, 0.0013322 , 0.00200272, 0.00165311, 0.00133212]),
 'std_score_time': array([6.26565729e-04, 4.70134207e-04, 2.24783192e-07, 4.70471865e-04,
        9.28298393e-04, 5.94720425e-07, 4.68565457e-04, 1.26474259e-05,
        4.63703822e-04, 4.71821366e-04]),
 'param_n_neighbors': masked_array(data=[2, 2, 3, 3, 4, 4, 5, 5, 6, 6],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_weights': masked_array(data=['uniform', 'distance', 'uniform', 'distance',
            

### Prove that best_score_ is the mean of all the k-fold scores
Here's a little check to see how best_score_ is derived from cv_results_

In [7]:
# Get the index of the best hyperparameter combination chosen by GridSearchCv()
grid.best_index_

3

In [8]:
# Get the mean and std of the k-fold scores for the best hyperparameter combination
print(grid.cv_results_['mean_test_score'][grid.best_index_])
print(grid.cv_results_['std_test_score'][grid.best_index_])

0.7711643653097031
0.006410593055068143
